<a href="https://colab.research.google.com/github/Khoan-IT/TrainingWithTPU/blob/main/ResNet50_GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# plot diagnostic learning curves
def summarize_diagnostics(history):
	plt.figure(figsize=(8, 8))
	plt.suptitle('Training Curves')
	# plot loss
	plt.subplot(211)
	plt.title('Cross Entropy Loss')
	plt.plot(history.history['loss'], color='blue', label='train')
	plt.plot(history.history['val_loss'], color='orange', label='val')
	plt.legend(loc='upper right')
	# plot accuracy
	plt.subplot(212)
	plt.title('Classification Accuracy')
	plt.plot(history.history['accuracy'], color='blue', label='train')
	plt.plot(history.history['val_accuracy'], color='orange', label='val')
	plt.legend(loc='lower right')
	return plt
 
# print test set evaluation metrics
def model_evaluation(model, evaluation_steps,x_test,y_test):
	print('\nTest set evaluation metrics')
	loss0,accuracy0 = model.evaluate(x_test,y_test, steps = evaluation_steps)
	print("loss: {:.2f}".format(loss0))
	print("accuracy: {:.2f}".format(accuracy0))

def model_report(model, history, x_test,y_test,evaluation_steps = 10):
	plt = summarize_diagnostics(history)
	plt.show()
	#model_evaluation(model, evaluation_steps,x_test,y_test)

In [ ]:
from keras.applications.resnet50 import ResNet50
from keras.datasets import cifar100
import tensorflow as tf
import datetime
import matplotlib.pyplot as plt
import numpy as np

VALIDATION_SPLIT = 0.2
TARGET_SIZE = [32,32]

(x_train, y_train), (x_test, y_test) = cifar100.load_data(label_mode='fine')
data_size = len(x_train)
temp = []
for i in range(data_size):
  one_hot = np.eye(1,100,int(y_train[i]),dtype = np.int64)
  temp.append(one_hot.tolist()[0])
y_train = temp
x_train, x_test = x_train / 255, x_test / 255 # normalize data
x_val = x_train[-int(VALIDATION_SPLIT*len(y_train)):]
y_val = y_train[-int(VALIDATION_SPLIT*len(y_train)):]
x_train = x_train[:int(len(y_train)-VALIDATION_SPLIT*len(y_train))]
y_train = y_train[:int(len(y_train)-VALIDATION_SPLIT*len(y_train))]
print(len(x_train))
print(len(x_val))
BATCH_SIZE = 256
AUTOTUNE = tf.data.experimental.AUTOTUNE # https://www.tensorflow.org/guide/data_performance

def _input_fn(x,y, BATCH_SIZE,train=False):
  ds = tf.data.Dataset.from_tensor_slices((x,y))
  ds = ds.shuffle(buffer_size=data_size)
  if train:
    ds = ds.repeat()
  ds = ds.batch(BATCH_SIZE)
  ds = ds.prefetch(buffer_size=AUTOTUNE)
  return ds

train_ds =_input_fn(x_train,y_train, BATCH_SIZE,train=True) #PrefetchDataset object
validation_ds =_input_fn(x_val,y_val, BATCH_SIZE, train = False) #PrefetchDataset object
test_ds =_input_fn(x_test,y_test, BATCH_SIZE) #PrefetchDataset object

40000
10000


In [ ]:
model = tf.keras.applications.ResNet50(weights=None, input_shape=[*TARGET_SIZE, 3], classes=100)
model.compile(
  optimizer='adam',
  loss='categorical_crossentropy',
  metrics=['accuracy'])

model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 38, 38, 3)    0           input_8[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 16, 16, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 16, 16, 64)   256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [ ]:
from time import process_time, time
t1_start = time() 
his = model.fit(train_ds,
    epochs = 25,
    validation_data = validation_ds,
    validation_steps = 39,
    steps_per_epoch = 156,
    verbose=True
)

t1_stop = time()    
print("Elapsed time during the whole program in seconds model", t1_stop-t1_start) 

Epoch 1/25
156/156 [==============================] - 28s 123ms/step - loss: 4.9459 - accuracy: 0.0551 - val_loss: 5.2867 - val_accuracy: 0.0102
Epoch 2/25
156/156 [==============================] - 19s 119ms/step - loss: 4.0741 - accuracy: 0.1219 - val_loss: 8.4846 - val_accuracy: 0.0108
Epoch 3/25
156/156 [==============================] - 18s 116ms/step - loss: 3.8663 - accuracy: 0.1532 - val_loss: 49.0207 - val_accuracy: 0.0441
Epoch 4/25
156/156 [==============================] - 18s 115ms/step - loss: 3.5632 - accuracy: 0.1891 - val_loss: 64.2079 - val_accuracy: 0.0095
Epoch 5/25
156/156 [==============================] - 18s 114ms/step - loss: 3.5302 - accuracy: 0.1923 - val_loss: 3.9762 - val_accuracy: 0.1064
Epoch 6/25
156/156 [==============================] - 18s 116ms/step - loss: 3.1437 - accuracy: 0.2492 - val_loss: 3.2451 - val_accuracy: 0.2169
Epoch 7/25
156/156 [==============================] - 18s 116ms/step - loss: 3.0439 - accuracy: 0.2766 - val_loss: 6.2890 - val_